
## TECHIN513:  Lab 1 - Elementary Music Synthesis

**Team Members**: 

**Overview**

The purpose of this lab is to familiarize you with constructing and processing discrete-time audio signals. You will review how to synthesize music notes and play them. You will concatenate a series of music notes into a small music piece, add volume variation to the music piece, overlap the adjacent notes to further make the music smoother and more realistic.

Through this process, you will learn to perform the following sound synthesis tasks:
- Generating musical notes
- Improving the quality of the generated sound
- Overlapping notes

In [ ]:
import numpy as np
import IPython.display as ipd
# Usage:
# ipd.Audio(audio_array, rate=sampling_rate, autoplay=True)
import matplotlib.pyplot as plt
from scipy.signal import hilbert
from scipy.io import wavfile as wav

In [84]:
# SimpleAudio constants.
DEFAULT_SAMPLE_RATE = 8000
DEFAULT_AMPLITUDE = 1
DEFAULT_BYTES_PER_SAMPLE = 2
DEFAULT_NUM_CHANNELS = 1

# Musical note frequencies.
A = 220.0
B = A * 2 ** (2 / 12)
C = A * 2 ** (3 / 12)
E = A * 2 ** (7 / 12)

In [ ]:
def play(data, num_channels=DEFAULT_NUM_CHANNELS, bytes_per_sample=DEFAULT_BYTES_PER_SAMPLE,
         sample_rate=DEFAULT_SAMPLE_RATE):
    """Returns a playable audio object

    Args:
        data: The audio data to be played (as a NumPy array).
        num_channels: The number of audio channels.
        bytes_per_sample: The number of bytes per sample per channel.
        sample_rate: The sample rate.
    """
    # Enter code here
    

def display_envelope(signal, duration=-1, sample_rate=DEFAULT_SAMPLE_RATE, bytes_per_sample=DEFAULT_BYTES_PER_SAMPLE):

    """
    Plots the envelope of a duration of the generated audio. 

    Args:
        signal: The audio signal to be displayed (as a NumPy array).
        duration: The length of hte signal to be displayed. -1 to display entire signal.
        num_channels: The number of audio channels.
        bytes_per_sample: The number of bytes per sample per channel.
        sample_rate: The sample rate.
    """
    
    if duration < 0:
        duration = len(signal)/sample_rate
    
    n = int(duration*sample_rate)
    
    analytic_signal = hilbert(signal[0:n])
    amplitude_envelope = np.abs(analytic_signal)
        
    t = np.linspace(0,duration,len(amplitude_envelope))
    
    plt.figure()    
    plt.plot(t,amplitude_envelope)
    plt.show()

## Task 1: Generating Musical Notes

In [86]:
# defining note function 
def note(frequency, duration, amplitude=DEFAULT_AMPLITUDE, sample_rate=DEFAULT_SAMPLE_RATE):
    """Generates samples of a musical note.

    Samples are generated according to following equation:

        y(t) = A sin(wt)

    Args:
        frequency: The frequency of the note.
        duration: The length of the note in seconds.
        amplitude: The amplitude of the note.
        sample_rate: The sample rate.

    Returns:
        An array of samples representing the note.
    """
    # Enter code here

In [90]:
# Use your function to generate the musical score for the Scarborough Fair.


# Play the audio sample

## Task 2: Time Scaling and Time Shifting Audio Signals

In [ ]:
ipd.Audio('train32.wav')

In [ ]:
fs1, w = wav.read('train32.wav')                      # Load audio file
print('Sampling rate: ' + str(fs1) + 'Hz')                # Extract sampling frequency

plt.figure()    
_ = plt.plot(np.arange(np.size(w)), w)
plt.xlabel("Time Index")


In [ ]:
# your code of task 2 goes below

def time_scale(signal, a, fs):
    """
    Perform time scaling on an audio waveform.

    Args:
        signal (numpy array): Input numpy array.
        a (float): Time scaling factor.
        fs (int): Sampling frequency (used to handle time shift in seconds if needed).
    
    Returns:
        numpy array: Transformed waveform.
    """
    n = np.arange(len(signal))  # Original time indices
    
    if a > 1: # signal compression
        # your code here
    elif a < 1 and a > 0: # signal expansion
        # your code here
    elif a == 1:
        transformed_signal = signal
    else:
        print("Invalid choice of a")

    return transformed_signal


In [ ]:
# your code goes here

In [ ]:

def time_shift(signal, b):
    """
    Perform time shifting on a signal.

    Args:
        signal (numpy array): Input numpy array.
        b (int): Time shifting factor.
        fs (int): Sampling frequency (used to handle time shift in seconds if needed).
    
    Returns:
        numpy array: Transformed waveform.
    """

    # Check if `b` is a valid integer
    if not isinstance(b, int):
        raise ValueError("Invalid choice of b.")
    
    # Perform time shifting
    transformed_signal = np.roll(signal, b)
    
    # Handle zero-padding for positive or negative shifts
    if b > 0:
        transformed_signal[:b] = 0  # Zero-pad the beginning
    elif b < 0:
        transformed_signal[b:] = 0  # Zero-pad the end

    return transformed_signal

In [ ]:
# your code goes here

## Task 3: Volume Variations

In [6]:
# Volume Variation Function Attack, Decay, Sustain and Release (ADSR)
def adsr(note):
    """Generates an ADSR envelope and applies it to a note.

        A: Linear increase to 60% amplitude over 20% length.
        D: Linear decrease to 50% amplitude over 4% length.
        S: Constant 50% amplitude over 40% length.
        R: Linear decrease to 0% amplitude over 36% length.

    Args:
        note: The note to be modified.

    Returns:
        A note that has been scaled by the ADSR envelope.
    """
    # Enter code here

In [7]:
# Use your function Apply an ADSR envelope to each note.


# Play the audio sample


# Display the envelope

**Supplementary Questions:** Instead of enveloping with the ADSR function, how would you envelope with a decaying exponential?

## Task 4: Overlapping Notes

In [8]:
def overlap(notes, offset):
    """Joins notes together with overlap between consecutive notes.

    Args:
        notes: An array of notes to be joined.
        offset: The amount of allowable overlap between consecutive notes.

    Returns:
        A score made from the input notes.
    """
    # Enter code here

In [18]:
# Smooth the sound by overlapping the notes.


# Play the audio sample


# Display the envelope